In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Congestive_heart_failure"
cohort = "GSE182600"

# Input paths
in_trait_dir = "../../input/GEO/Congestive_heart_failure"
in_cohort_dir = "../../input/GEO/Congestive_heart_failure/GSE182600"

# Output paths
out_data_file = "../../output/preprocess/Congestive_heart_failure/GSE182600.csv"
out_gene_data_file = "../../output/preprocess/Congestive_heart_failure/gene_data/GSE182600.csv"
out_clinical_data_file = "../../output/preprocess/Congestive_heart_failure/clinical_data/GSE182600.csv"
json_path = "../../output/preprocess/Congestive_heart_failure/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene Expression of Cardiogenic Shock Patients under Extracorporeal Membrane Oxygenation"
!Series_summary	"Prognosis for cardiogenic shock patients under ECMO was our study goal. Success defined as survived more than 7 days after ECMO installation and failure died or had multiple organ failure in 7 days. Total 34 cases were enrolled, 17 success and 17 failure."
!Series_summary	"Peripheral blood mononuclear cells collected at ECMO installation 0hr, 2hr and removal were used analyzed."
!Series_overall_design	"Analysis of the cardiogenic shock patients at extracorporeal membrane oxygenation treatment by genome-wide gene expression. Transcriptomic profiling between successful and failure groups were analyzed."
Sample Characteristics Dictionary:
{0: ['disease state: Acute myocarditis', 'disease state: Acute myocardial infarction', 'disease state: Dilated cardiomyopathy, DCMP', 'disease state: Congestive heart failure', 'disease state: Dilated cardiomyop

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# For trait (Congestive heart failure)
# Looking at the sample characteristics, outcome (row 3) indicates success or failure of ECMO treatment
# which is directly related to the trait of congestive heart failure
trait_row = 3

def convert_trait(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 1 for failure (worse outcome), 0 for success
    if value.lower() in ['failure']:
        return 1
    elif value.lower() == 'success':
        return 0
    return None

# For age
age_row = 1

def convert_age(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

# For gender
gender_row = 2

def convert_gender(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 0 for female, 1 for male
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    return None

# 3. Save Metadata
# Initial filtering - check if both gene and trait data are available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# I'll skip this part for now as we don't have the proper clinical data structure
# This will be handled in the next step when we have the appropriate data format


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Congestive_heart_failure/GSE182600/GSE182600_series_matrix.txt.gz


Gene data shape: (29363, 78)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238',
       'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268',
       'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286',
       'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers start with 'ILMN_', which indicates these are Illumina array probe IDs,
# not standard human gene symbols. These IDs need to be mapped to official gene symbols
# for proper biological interpretation.

# Illumina probe IDs like ILMN_1343291 need to be converted to their corresponding gene symbols
# using annotation information typically provided in platform files or through bioinformatics
# databases.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'Transcript', 'Species', 'Source', 'Search_Key', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC

Column 'ID' contains values matching gene data ID pattern


Column 'Transcript' contains values matching gene data ID pattern


Column 'Species' contains values matching gene data ID pattern


Column 'Source' contains values matching gene data ID pattern



Checking for columns containing transcript or gene related terms:
Column 'Transcript' may contain gene-related information
Sample values: ['ILMN_333737', 'ILMN_333646', 'ILMN_333584']
Column 'ILMN_Gene' may contain gene-related information
Sample values: ['ERCC-00162', 'ERCC-00071', 'ERCC-00009']
Column 'Entrez_Gene_ID' may contain gene-related information
Sample values: [nan, nan, nan]
Column 'Symbol' may contain gene-related information
Sample values: ['ERCC-00162', 'ERCC-00071', 'ERCC-00009']


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in gene_annotation that correspond to gene identifiers and gene symbols
# From the previous output, we can see:
# - The 'ID' column contains values matching the ILMN_ pattern used in gene_data index
# - The 'Symbol' column appears to contain gene symbols

# 2. Get a gene mapping dataframe using the get_gene_mapping function
probe_col = 'ID'
gene_col = 'Symbol'
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

# Preview the mapping dataframe
print("Gene mapping dataframe sample:")
print(gene_mapping.head())
print(f"Mapping dataframe shape: {gene_mapping.shape}")

# 3. Apply the gene mapping to convert probe-level data to gene-level data
# The apply_gene_mapping function handles the many-to-many mapping as specified
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print information about the resulting gene expression dataframe
print(f"\nGene expression data shape after mapping: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10].tolist())

# Save the gene data to a CSV file
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


Gene mapping dataframe sample:
             ID        Gene
0  ILMN_3166687  ERCC-00162
1  ILMN_3165566  ERCC-00071
2  ILMN_3164811  ERCC-00009
3  ILMN_3165363  ERCC-00053
4  ILMN_3166511  ERCC-00144
Mapping dataframe shape: (29377, 2)



Gene expression data shape after mapping: (20206, 78)
First 10 gene symbols after mapping:
['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT']


Gene expression data saved to: ../../output/preprocess/Congestive_heart_failure/gene_data/GSE182600.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        # Check if trait is biased
        trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
        if trait_type == "binary":
            is_biased = judge_binary_variable_biased(linked_data, trait)
        else:
            is_biased = judge_continuous_variable_biased(linked_data, trait)
            
        # Remove biased demographic features
        if "Age" in linked_data.columns:
            age_biased = judge_continuous_variable_biased(linked_data, 'Age')
            if age_biased:
                linked_data = linked_data.drop(columns='Age')
                
        if "Gender" in linked_data.columns:
            gender_biased = judge_binary_variable_biased(linked_data, 'Gender')
            if gender_biased:
                linked_data = linked_data.drop(columns='Gender')
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to atrial fibrillation after cardiac surgery, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for atrial fibrillation after cardiac surgery, which is relevant to arrhythmia research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for arrhythmia analysis."
    )
    
    print("Dataset is not usable for arrhythmia analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (20206, 78)
Gene data shape after normalization: (19445, 78)


Normalized gene expression data saved to ../../output/preprocess/Congestive_heart_failure/gene_data/GSE182600.csv
Original clinical data preview:
         !Sample_geo_accession                        GSM5532093  \
0  !Sample_characteristics_ch1  disease state: Acute myocarditis   
1  !Sample_characteristics_ch1                         age: 33.4   
2  !Sample_characteristics_ch1                         gender: F   
3  !Sample_characteristics_ch1                  outcome: Success   
4  !Sample_characteristics_ch1                   cell type: PBMC   

                         GSM5532094                        GSM5532095  \
0  disease state: Acute myocarditis  disease state: Acute myocarditis   
1                         age: 51.2                         age: 51.9   
2                         gender: M                         gender: F   
3                  outcome: Success                  outcome: Failure   
4                   cell type: PBMC                   cell type: PBMC   

      

Data shape after handling missing values: (78, 19448)
For the feature 'Congestive_heart_failure', the least common label is '1.0' with 31 occurrences. This represents 39.74% of the dataset.
Quartiles for 'Age':
  25%: 47.0
  50% (Median): 52.15
  75%: 56.35
Min: 16.1
Max: 78.9
For the feature 'Gender', the least common label is '0.0' with 24 occurrences. This represents 30.77% of the dataset.
A new JSON file was created at: ../../output/preprocess/Congestive_heart_failure/cohort_info.json


Linked data saved to ../../output/preprocess/Congestive_heart_failure/GSE182600.csv
